In [ ]:
print(sys.executable)

In [ ]:
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import matplotlib.ticker as mtick
from matplotlib.cm import get_cmap
import addcopyfighandler

from IPython.core.display import HTML
from IPython.display import display

import model2 as mod
import model_fsolve as modf

In [ ]:
import fluids as fds

In [ ]:
par = {}

par["eps"] = 0.001

par["ref"] = 1

par["rho"] = 997 # kg/m3
par["nu"] = 0.896*1e-6 # at 25°C, in m2/s https://www4.ac-nancy-metz.fr/physique/ancien_site/Tp-phys/Term/TP-fluid/visco-eau.htm
par["eta"] = par["rho"]*par["nu"]

In [ ]:
# Heat exchanger inputs --------------------------------------------------

# REAL SPRING
# -----------

par["N"] = 165

par["Lx"] = 1.39985 # longueur d'un canal en m, not used in the row calculation
par["Ly"] = 0.0008157 # distance inter canaux en m

# pour le SPRING, avec 1 mm d'écart entre chaque duo de risers, et 5 mm de largeur par riser, ça fait 989 mm de largeur

par["h_man"] = 0.010 # SPRING en vrai c'est 0.010
par["l_man"] = 0.022

par["Dx"] = 0.00476 # m, not used in the row calculation
par["Din"] = (2*par["h_man"]*par["l_man"])/(par["h_man"]+par["l_man"]) # m
par["Dout"] = par["Din"] # m

par["theta"] = 90.

par["rough"] = 0.0015 # PVC/plastic pipe absolute roughness is 0.0015

In [ ]:
# Manifold tubulaire

par["Ax"] = math.pi*(par["Dx"]/2)**2
par["Ain"] = math.pi*(par["Din"]/2)**2
par["Aout"] = math.pi*(par["Dout"]/2)**2

par["a_x"] = 84.7/800
# par["a"] = 0.
par["b_x"] = 5.36/800

# dP (Pa) = (rho/2) (a_x u**2 + b_x u) its the pressure loss function of a heat exchanger

par["sch"] = "exchanger"

In [ ]:
# Manifold rectangulaire (SPRING ou autre)

par["Ax"] = math.pi*(par["Dx"]/2)**2
par["Ain"] = par["h_man"]*par["l_man"]
par["Aout"] = par["Ain"]

par["a_x"] = 84.7/800
# par["a"] = 0.
par["b_x"] = 5.36/800

# dP (Pa) = (rho/2) (a_x u**2 + b_x u) its the pressure loss function of a heat exchanger

par["sch"] = "exchanger"

In [ ]:
# 'Crane' or 'perso'
par['method'] = 'Crane'

# Listes de taille par["N"]

par['Kxin'] = []
par['Kyin'] = []
par['Kxout'] = []
par['Kyout'] = []

In [ ]:
# Test d'un seul débit

par["QF"] = 100/3600000 # m3/s (0.000278 m3/s = 1000 L/h) (le premier nombre est en L/h)
# Speed and Reynolds at inlet manifold
par["U"] = par["QF"]/par["Ain"]
par["Reman"] = par["U"]*(par["rho"]*par["Din"])/par["eta"]
tabl,res = modf.PL_fsolve(par,par["sch"],True)

In [ ]:
# Liste de plusieurs débits

# list_k = np.linspace(1e-6,5*1e-6,10)
# list_k = np.linspace(1e-8,1e-6,20)
list_Q = [10/3600000,50/3600000,100/3600000,150/3600000,200/3600000] # m3/s
list_Q_L = 3600000*np.array(list_Q)

list_PL = []
list_mn = []
list_std = []
list_tabl = []

In [ ]:
print(list_Q)
print(list_Q_L)

In [ ]:
# Model fsolve

for Q in list_Q:
    print(Q)
    par["QF"] = Q
    # Speed and Reynolds at inlet manifold
    par["U"] = par["QF"]/par["Ain"]
    par["Reman"] = par["U"]*(par["rho"]*par["Din"])/par["eta"]

    tabl,res = modf.PL_fsolve(par,par["sch"],False)
    list_PL.append(res)
    list_tabl.append(tabl)

    list_mn.append(tabl['qx'].mean()) # fow rate qx is in L/h
    list_std.append(tabl['qx'].std())


In [ ]:
rho = par['rho']

for q in range(len(list_Q)):
    PLq = list_PL[q]

    lin_in = []
    lin_x = []
    lin_out = []
    lin_in_cum = []
    lin_out_cum = []
    sing = []

    tab = list_tabl[q]

    Qin = []
    Qout = []
    
    for i in range(par["N"]):
        Qin.append(sum([tab['qx'][j] for j in range(0,i+1)]))
        Qout.append(sum([tab['qx'][j] for j in range(i,par["N"])]))

    tab['Qin'] = Qin
    tab['Qout'] = Qout

    for i in range(par["N"]):
        Qin_i = tab['Qin'][i]
        Qout_i = tab['Qout'][i]
        qx_i = tab['qx'][i]

        uin_i = (Qin_i/3600000)/par['Ain']
        ux_i = (qx_i/3600000)/par['Ax']
        uout_i = (Qout_i/3600000)/par['Aout']

        Rein_i = fds.core.Reynolds(uin_i,par['Din'],par['rho'],mu=par['eta'])
        Rex_i = fds.core.Reynolds(ux_i,par['Dx'],par['rho'],mu=par['eta'])
        Reout_i = fds.core.Reynolds(uout_i,par['Dout'],par['rho'],mu=par['eta'])
        fin_i = fds.friction.friction_factor(Re = Rein_i)
        fx_i = fds.friction.friction_factor(Re = Rex_i)
        fout_i = fds.friction.friction_factor(Re = Reout_i)

        ain_i = fin_i*(par['Ly']/par['Din'])
        ax_i = fx_i*(par['Lx']/par['Dx'])
        aout_i = fout_i*(par['Ly']/par['Dout'])

        lin_in.append((rho/2)*ain_i*uin_i**2)
        lin_x.append((rho/2)*ax_i*ux_i**2)
        lin_out.append((rho/2)*aout_i*uout_i**2)

    tab['lin_in'] = lin_in
    tab['lin_x'] = lin_x
    tab['lin_out'] = lin_out

    for i in range(par["N"]):
        lin_in_cum.append(sum([lin_in[j] for j in range(i,par["N"])]))
        lin_out_cum.append(sum([lin_out[j] for j in range(0,i+1)]))

    tab['lin_in_cum'] = lin_in_cum
    tab['lin_out_cum'] = lin_out_cum

    for i in range(par["N"]):
        sing.append(PLq-tab['lin_in_cum'][i]-tab['lin_x'][i]-tab['lin_out_cum'][i])
    
    tab['sing'] = sing

In [ ]:
sum([list_tabl[2]['qx'][i] for i in range(par["N"])])

In [ ]:
print(list_PL)
print(list_tabl[0])

In [ ]:
q = 2
Q = list_Q_L[q]

x = np.array(range(0,17))
x = 10*x

In [ ]:
print(x)
print(len(x))

In [ ]:
# Stacked bars chart

labels = [str(x[i]) for i in range(len(x))]

tab = list_tabl[q]

lin_in_cum = []
lin_x = []
lin_out_cum = []
sing = []

for i in range(len(x)):
    lin_in_cum.append(tab['lin_in_cum'][x[i]])
    lin_x.append(tab['lin_x'][x[i]])
    lin_out_cum.append(tab['lin_out_cum'][x[i]])
    sing.append(tab['sing'][x[i]])

In [ ]:
lin_in_cum = np.array(lin_in_cum)
lin_x = np.array(lin_x)
lin_out_cum = np.array(lin_out_cum)
sing = np.array(sing)

width = 0.35       # the width of the bars: can also be len(x) sequence

fig, ax = plt.subplots()

ax.bar(labels, lin_in_cum, width, label='Inlet header linear PL')
ax.bar(labels, lin_x, width, bottom=lin_in_cum,label='Riser linear PL')
ax.bar(labels, lin_out_cum, width, bottom=lin_in_cum+lin_x,label='Outlet header linear PL')
ax.bar(labels, sing, width, bottom=lin_in_cum+lin_x+lin_out_cum,label='Singular PL')

ax.plot(labels,np.array(17*[list_PL[q]]))

ax.set_ylabel('PL (Pa)')
# ax.set_title('SPRING')
ax.legend()

plt.show()

In [ ]:
print(np.average(sing))
print(np.average(sing)/list_PL[q])
print(np.average(lin_x))
print(np.average(lin_x)/list_PL[q])

In [ ]:
list_tabl[0]['qx']

In [ ]:
# Inputs for singular pressure losses

d_epdm = 0.015
d_red = 0.0078 # 0.011 en pratique, diminué à 0.0078 pour que les PL + 2m EPDM fit aux mesures exp Technoptic avec SSA
d_man = 0.0081 # 13 mm de base, quand le manifold ne fait plus que 5 mm de hauteur c'est 8,1 mm de diamètre hydraulique

L_epdm = 2.

angle_bm_epdm = 90.
angle_bm_man = 90.

In [ ]:
list_Re = []

Kc_list_R = []
Kc_list_H = []
PL_c_R = []
PL_c_H = []

Kdm_list_R = []
Kdm_list_H = []
PL_dm_R = []
PL_dm_H = []

Kcm_list_R = []
Kcm_list_H = []
PL_cm_R = []
PL_cm_H = []

Kd_list_R = []
Kd_list_H = []
PL_d_R = []
PL_d_H = []

list_Kbm_man = []
PL_bm_man = []

list_Kbm_epdm = []
PL_bm_epdm = []

PL_epdm = []

for Q in list_Q:
    u_epdm = Q/(math.pi*(d_epdm/2)**2)
    u_red = Q/(math.pi*(d_red/2)**2)
    u_man = Q/(math.pi*(d_man/2)**2)

    # Reynolds dans les liaisons EPDM, dans la réduction du QDF, dans les manifolds
    Re_epdm = fds.core.Reynolds(u_epdm,d_epdm,par["rho"],mu=par["eta"])
    list_Re.append(Re_epdm)
    Re_red = fds.core.Reynolds(u_red,d_red,par["rho"],mu=par["eta"])
    Re_man = fds.core.Reynolds(u_man,d_man,par["rho"],mu=par["eta"])

    # Contraction sharp EPDM-reduction
    Kc_H = fds.fittings.contraction_sharp(d_epdm,d_red,Re=Re_epdm,roughness=par["rough"],method='Hooper')
    Kc_R = fds.fittings.contraction_sharp(d_epdm,d_red,Re=Re_epdm,roughness=par["rough"],method='Rennels')
    Kc_list_H.append(Kc_H)
    Kc_list_R.append(Kc_R)
    PL_c_H.append((par["rho"]/2)*Kc_H*u_epdm**2)
    PL_c_R.append((par["rho"]/2)*Kc_R*u_epdm**2)

    # Diffuser sharp reduction-man
    Kdm_H = fds.fittings.diffuser_sharp(d_red,d_man,Re=Re_red,roughness=par["rough"],method='Hooper')
    Kdm_R = fds.fittings.diffuser_sharp(d_red,d_man,Re=Re_red,roughness=par["rough"],method='Rennels')
    Kdm_list_H.append(Kdm_H)
    Kdm_list_R.append(Kdm_R)
    PL_dm_H.append((par["rho"]/2)*Kdm_H*u_red**2)
    PL_dm_R.append((par["rho"]/2)*Kdm_R*u_red**2)

    # Contraction sharp manifold-reduction

    Kcm_H = fds.fittings.contraction_sharp(d_man,d_red,Re=Re_man,roughness=par["rough"],method='Hooper')
    Kcm_R = fds.fittings.contraction_sharp(d_man,d_red,Re=Re_man,roughness=par["rough"],method='Rennels')
    Kcm_list_H.append(Kcm_H)
    Kcm_list_R.append(Kcm_R)
    PL_cm_H.append((par["rho"]/2)*Kcm_H*u_man**2)
    PL_cm_R.append((par["rho"]/2)*Kcm_R*u_man**2)

    # Diffuser sharp reduction-EPDM
    Kd_H = fds.fittings.diffuser_sharp(d_red,d_epdm,Re=Re_red,roughness=par["rough"],method='Hooper')
    Kd_R = fds.fittings.diffuser_sharp(d_red,d_epdm,Re=Re_red,roughness=par["rough"],method='Rennels')
    Kd_list_H.append(Kd_H)
    Kd_list_R.append(Kd_R)
    PL_d_H.append((par["rho"]/2)*Kd_H*u_red**2)
    PL_d_R.append((par["rho"]/2)*Kd_R*u_red**2)

    # Bend miter in the EPDM
    Kbm_epdm = fds.fittings.bend_miter(angle_bm_epdm, Di=d_epdm, Re=fds.core.Reynolds(u_epdm,d_epdm,par["rho"],mu=par["eta"]),roughness=par["rough"], L_unimpeded=d_epdm, method='Rennels')
    list_Kbm_epdm.append(Kbm_epdm)
    PL_bm_epdm.append((par["rho"]/2)*Kbm_epdm*u_epdm**2)

    # Bend miter in the manifold
    Kbm = fds.fittings.bend_miter(angle_bm_man, Di=d_man, Re=Re_man, L_unimpeded=2*d_man, method='Rennels')
    list_Kbm_man.append(Kbm)
    PL_bm_man.append((par["rho"]/2)*Kbm*u_man**2)

    # Linear pressure losses

    f_epdm = fds.friction.friction_factor(Re = Re_epdm,eD=par["rough"]/d_epdm)
    K_lin_epdm = f_epdm*(L_epdm/d_epdm)
    PL_epdm.append((par["rho"]/2)*K_lin_epdm*u_epdm**2)

# Total pressure losses

exp_PL_tot = [(3.3*1E-5*Q**2+0.002363466*Q)*1000 for Q in list_Q_L]

PL_tot = np.array(PL_c_H) + np.array(PL_dm_H) + np.array(PL_d_H) + np.array(PL_cm_H) + 2*np.array(PL_bm_epdm) + 2*np.array(PL_bm_man) + np.array(list_PL)
PL_tot_lin = PL_tot + np.array(PL_epdm)

print(list_Q_L)
print(exp_PL_tot)
print(PL_tot)

plt.plot(list_Q_L,list_Re,label='Reynolds in EPDM')
plt.legend()
plt.show()

plt.plot(list_Q_L,PL_c_R,label='Rennels')
plt.plot(list_Q_L,PL_c_H,label='Hooper')
plt.legend()
plt.show()

plt.plot(list_Q_L,PL_d_R,label='Rennels')
plt.plot(list_Q_L,PL_d_H,label='Hooper')
plt.legend()
plt.show()

plt.plot(list_Q_L,PL_bm_epdm,label='Rennels bm_epdm')
plt.legend()
plt.show()

plt.plot(list_Q_L,PL_bm_man,label='Rennels bm_man')
plt.legend()
plt.show()

plt.plot(list_Q_L,PL_epdm,label='Linear pressure losses')
plt.legend()
plt.show()

plt.plot(list_Q_L,PL_tot,label='Estimation of SPRING PL')
plt.plot(list_Q_L,PL_tot_lin,label='Estimation + 2 m of EPDM')
plt.plot(list_Q_L,exp_PL_tot,label='Measured SPRING PL')
plt.legend()
plt.xlabel('Q (L/h)')
plt.ylabel('PL (Pa)')
plt.grid()
plt.show()


In [ ]:
# Stacked bars chart

labels = ['10', '50', '100', '150', '200']

# cont_diff = np.array(PL_c_R) + np.array(PL_dm_R) + np.array(PL_d_R) + np.array(PL_cm_R)
cont_diff = np.array(PL_c_H) + np.array(PL_dm_H) + np.array(PL_d_H) + np.array(PL_cm_H)
bends = 2*np.array(PL_bm_epdm) + 2*np.array(PL_bm_man)
harp = np.array(list_PL)
reg = np.array(PL_epdm)

width = 0.35       # the width of the bars: can also be len(x) sequence

fig, ax = plt.subplots()

ax.bar(labels, cont_diff, width, label='Sharp contractions and diffusers')
ax.bar(labels, bends, width, bottom=cont_diff,label='Bends (x4)')
ax.bar(labels, harp, width, bottom=cont_diff+bends,label='Harp exchanger')
ax.bar(labels, reg, width, bottom=cont_diff+bends+harp,label='Linear PL')

ax.set_ylabel('PL (Pa)')
ax.set_title('SPRING')
ax.legend()

plt.show()

In [ ]:
save = [PL_tot,PL_tot_lin,exp_PL_tot,cont_diff,bends,harp,reg]

In [ ]:
save2 = [PL_tot,PL_tot_lin,exp_PL_tot,cont_diff,bends,harp,reg]

In [ ]:
ratio = (PL_tot_lin-save[1])/save[1]

plt.plot(list_Q_L,ratio)
plt.xlabel('Q (L/h)')
plt.ylabel('PL increase')
plt.title('SPRING h5 mm PL increase in % of SPRING h10 mm PL')
plt.gca().set_yticklabels([f'{x:.0%}' for x in plt.gca().get_yticks()]) 
plt.grid()
plt.show()

In [ ]:
plt.plot(list_Q_L,PL_tot,label='Total pressure losses')

plt.legend()
plt.xlabel('Q (L/h)')
plt.ylabel('Re')

plt.grid()

In [ ]:
print(list_Q_L)
print(list_PL)
print(list_mn)
print([list_Q_L[i]/par["N"] for i in range(len(list_Q_L))])

# df_res = pd.DataFrame([np.array(list_Q_L),np.array(list_PL)],columns = ['Q_L','PL (Pa)'])
# display(HTML(df_res.to_html()))  


In [ ]:
# Répartition des débits par canal (en valeur absolue)

list_Q_L_round = [round(num, 0) for num in list_Q_L]
risers = np.linspace(0,par["N"]-1,par["N"])

for i in range(len(list_tabl)):
    plt.plot(risers,np.array(list_tabl[i]['qx']),label=str(list_Q_L_round[i])+' L/h')
    plt.legend()

plt.xlabel('N° riser')
plt.ylabel('qx (L/h)')
# plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.grid()

plt.show()

In [ ]:
# Répartition des débits par canal (en valeur relative au débit moyen attendu)

for i in range(len(list_tabl)):
    plt.plot(risers,np.array(list_tabl[i]['qx'])/list_mn[i],label=str(list_Q_L_round[i])+' L/h')
    plt.legend()

plt.xlabel('N° riser')
plt.ylabel('qx/q_mean')
# plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.grid()

plt.show()

In [ ]:
# Plot pressure losses

plt.plot(np.array(list_Q_L),np.array(list_PL))

plt.xlabel('Q (L/h)')
plt.ylabel('PL (Pa)')
# plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.grid()

plt.show()

In [ ]:
# Plot mean and standard deviation of flow rate

plt.plot(np.array(list_Q_L),np.array(list_mn))
plt.plot(np.array(list_Q_L),np.array(list_std))

plt.xlabel('Q (L/h)')
plt.ylabel('Flow rate (L/h)')
# plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.grid()

plt.show()

In [ ]:
# Plot standard deviation of flow rate / mean flow rate

plt.plot(np.array(list_Q_L),np.array(list_std)/np.array(list_mn))

plt.xlabel('Q (L/h)')
plt.ylabel('Ratio of flow rate standard deviation out of mean flow rate')
# plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.grid()

plt.show()

In [ ]:
print(np.array(list_std))
print(np.array(list_mn))
print(np.array(list_std)/np.array(list_mn))

In [ ]:
df = pd.DataFrame([1,2,3,4],columns = ['numbers'])

In [ ]:
l = [2,2,2]
l = np.array(l)

In [ ]:
l/2

In [ ]:
np.array(list_tabl[0]['qx'])

In [ ]:
risers = np.linspace(0,165,166)

In [ ]:
len(np.array(list_tabl[i]['qx']))

In [ ]:
len(risers)

In [ ]:
print(list_Q_L)

In [ ]:
[round(num, 0) for num in list_Q_L]